In [1]:
import os, argparse
import cv2
import sys
import pandas as pd
import tensorflow as tf
import numpy as np
import datetime
import time
from IPython.display import display, clear_output

In [2]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [3]:
"""
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()
"""

"\nfrom tensorflow.python.client import device_lib\n\ndef get_available_gpus():\n    local_device_protos = device_lib.list_local_devices()\n    return [x.name for x in local_device_protos if x.device_type == 'GPU']\n\nget_available_gpus()\n"

In [4]:
sys.path.insert(1, '/p/project/joaiml/ingolfsson1/COVID_Net')

from data import process_image_file

In [5]:
df_test = pd.read_csv('/p/project/joaiml/ingolfsson1/COVID_Net/labels/test_COVIDx8A.txt', sep=' ', header=None)
#df_test.columns = ["id", "path", "condition", "source", "split"]

df_test.columns = ["id", "path", "condition", "source"]

In [6]:
df_test.head()

,id,path,condition,source
0,419639-003251,MIDRC-RICORD-1C-419639-003251-46647-0.png,COVID-19,ricord
1,419639-001464,MIDRC-RICORD-1C-419639-001464-39871-0.png,COVID-19,ricord
2,419639-000918,MIDRC-RICORD-1C-419639-000918-78965-0.png,COVID-19,ricord
3,419639-003318,MIDRC-RICORD-1C-419639-003318-64285-0.png,COVID-19,ricord
4,419639-001015,MIDRC-RICORD-1C-419639-001015-81591-0.png,COVID-19,ricord


In [7]:
df_test.shape

(400, 4)

In [8]:
"""
df_out = df_test.copy()
df_out['pred'] = 'unknown'
df_out['time'] = 'unknown'
"""

# this needs to be changed if continuing inference from a different run

#df_out_higher = pd.read_csv('/p/project/joaiml/ingolfsson1/COVID_Net/results/total_ehl_480_1636665405.csv', sep=',')

#df_out_higher.tail()

df_out_higher = pd.read_csv('/p/project/joaiml/ingolfsson1/COVID_Net/results/test_covidx_480_1637081973.csv', sep=',')

df_out_higher.tail()

,id,path,condition,source,pred,time
395,a77b32f6-1999-4cc6-8a0d-1ef7e53850f0,a77b32f6-1999-4cc6-8a0d-1ef7e53850f0.png,pneumonia,rsna,pneumonia,21.440743
396,41bf2042-53a2-44a8-9a29-55e643af5ac0,41bf2042-53a2-44a8-9a29-55e643af5ac0.png,pneumonia,rsna,pneumonia,22.142910
397,ec09ddb3-9b0b-4749-9e0d-f94a4a7e5fed,ec09ddb3-9b0b-4749-9e0d-f94a4a7e5fed.png,pneumonia,rsna,pneumonia,21.561357
398,abff4044-3c0a-458f-b0c7-04254be7fc0c,abff4044-3c0a-458f-b0c7-04254be7fc0c.png,pneumonia,rsna,pneumonia,21.433315
399,27b32500-40ef-4c14-ab88-ace5e9ceea27,27b32500-40ef-4c14-ab88-ace5e9ceea27.png,pneumonia,rsna,pneumonia,21.400557


In [9]:
"""
df_out = df_test.copy()
df_out['pred'] = 'unknown'
df_out['time'] = 'unknown'

df_out.to_csv('/p/project/joaiml/ingolfsson1/COVID_Net/results/covidx_test_base.csv', index=False)
"""

"\ndf_out = df_test.copy()\ndf_out['pred'] = 'unknown'\ndf_out['time'] = 'unknown'\n\ndf_out.to_csv('/p/project/joaiml/ingolfsson1/COVID_Net/results/covidx_test_base.csv', index=False)\n"

In [10]:
df_out_lower = pd.read_csv('/p/project/joaiml/ingolfsson1/COVID_Net/results/covidx_test_base.csv', sep=',')

df_out_lower.tail()

,id,path,condition,source,pred,time
395,a77b32f6-1999-4cc6-8a0d-1ef7e53850f0,a77b32f6-1999-4cc6-8a0d-1ef7e53850f0.png,pneumonia,rsna,unknown,unknown
396,41bf2042-53a2-44a8-9a29-55e643af5ac0,41bf2042-53a2-44a8-9a29-55e643af5ac0.png,pneumonia,rsna,unknown,unknown
397,ec09ddb3-9b0b-4749-9e0d-f94a4a7e5fed,ec09ddb3-9b0b-4749-9e0d-f94a4a7e5fed.png,pneumonia,rsna,unknown,unknown
398,abff4044-3c0a-458f-b0c7-04254be7fc0c,abff4044-3c0a-458f-b0c7-04254be7fc0c.png,pneumonia,rsna,unknown,unknown
399,27b32500-40ef-4c14-ab88-ace5e9ceea27,27b32500-40ef-4c14-ab88-ace5e9ceea27.png,pneumonia,rsna,unknown,unknown


In [11]:
def format_pred(out):
    return out.decode('utf-8').split('\n')[0].split(' ')[-1]

In [12]:
def predict(imagepath, weightspath, metaname, ckptname, in_tensorname, out_tensorname, input_size):

    top_percent = 0.08

    mapping = {'normal': 0, 'pneumonia': 1, 'COVID-19': 2}
    inv_mapping = {0: 'normal', 1: 'pneumonia', 2: 'COVID-19'}

    tf.reset_default_graph()

    #config = tf.ConfigProto(device_count={'GPU':1, 'CPU':1})
    #config.gpu_options.allow_growth=True
    #config.allow_soft_placement=True

    #sess = tf.Session(config=config)
    sess = tf.Session()
    tf.get_default_graph()
    saver = tf.train.import_meta_graph(os.path.join(weightspath, metaname))
    saver.restore(sess, os.path.join(weightspath, ckptname))

    graph = tf.get_default_graph()

    image_tensor = graph.get_tensor_by_name(in_tensorname)
    pred_tensor = graph.get_tensor_by_name(out_tensorname)

    x = process_image_file(imagepath, top_percent, input_size)
    x = x.astype('float32') / 255.0
    pred = sess.run(pred_tensor, feed_dict={image_tensor: np.expand_dims(x, axis=0)})

    prediction = inv_mapping[pred.argmax(axis=1)[0]]

    return prediction

In [13]:
"""
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

with tf.device('/device:GPU:0'):
    print(get_available_gpus())
"""

"\nfrom tensorflow.python.client import device_lib\n\ndef get_available_gpus():\n    local_device_protos = device_lib.list_local_devices()\n    return [x.name for x in local_device_protos if x.device_type == 'GPU']\n\nwith tf.device('/device:GPU:0'):\n    print(get_available_gpus())\n"

In [14]:
"""
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)

with tf.device('/GPU:1'):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)

sess = tf.Session(config=config)    

# Run on the GPU
print(sess.run(c))
"""

"\nconfig = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)\n\nwith tf.device('/GPU:1'):\n    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])\n    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])\n    c = tf.matmul(a, b)\n\nsess = tf.Session(config=config)    \n\n# Run on the GPU\nprint(sess.run(c))\n"

In [15]:
"""
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
display(sess.run(c))
"""

"\n# Creates a graph.\na = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')\nb = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')\nc = tf.matmul(a, b)\n# Creates a session with log_device_placement set to True.\nsess = tf.Session(config=tf.ConfigProto(log_device_placement=True))\n# Runs the op.\ndisplay(sess.run(c))\n"

In [16]:
"""
#tf.logging.set_verbosity(tf.logging.DEBUG)

# Creates a graph.
with tf.device('/GPU:1'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')

c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))
"""

"\n#tf.logging.set_verbosity(tf.logging.DEBUG)\n\n# Creates a graph.\nwith tf.device('/GPU:1'):\n    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')\n    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')\n\nc = tf.matmul(a, b)\n# Creates a session with log_device_placement set to True.\nsess = tf.Session(config=tf.ConfigProto(log_device_placement=True))\n# Runs the op.\nprint(sess.run(c))\n"

In [17]:
"""
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
"""

'\nimport tensorflow as tf\nif tf.test.gpu_device_name():\n    print(\'Default GPU Device: {}\'.format(tf.test.gpu_device_name()))\nelse:\n    print("Please install GPU version of TF")\n'

In [18]:
# Using CXR4-A model (480x480 dimensions)

df_out_480 = df_out_higher.copy()

total = 0
ts = int(datetime.datetime.now().timestamp())

weightspath = '/p/project/joaiml/ingolfsson1/COVID_Net/models/COVIDNet-CXR4-A'
metaname = 'model.meta'
ckptname = 'model-18540'
in_tensorname = 'input_1:0'
out_tensorname = 'norm_dense_1/Softmax:0'
input_size = 480

for _, image, cond, source in df_test.values:

    split = 'test'

    if df_out_480[df_out_480['path'] == image]['pred'].iloc[0] == 'unknown':

        complete_path = '/p/project/joaiml/ingolfsson1/COVID_Net/data/{}/{}'.format(split, image)

        t = time.time()
        pred = predict(complete_path, weightspath, metaname, ckptname, in_tensorname, out_tensorname, input_size)
        t = time.time() - t

        total = total + t

        clear_output(wait=True)
        display('Image: {}/{}'.format(split, image))
        display('Expected: {}'.format(cond))
        display('Predicted: {}'.format(pred))
        display('Inference time: {}'.format(t))
        display('Total time elapsed: {}'.format(total))

        df_out_480['pred'] = np.where(df_out_480['path'] == image, pred, df_out_480['pred'])
        df_out_480['time'] = np.where(df_out_480['path'] == image, t, df_out_480['time'])

        df_out_480.to_csv('/p/project/joaiml/ingolfsson1/COVID_Net/results/test_covidx_480_{}.csv'.format(ts), index=False)

clear_output(wait=True)
display('Inference complete.')
display('Total time elapsed: {}'.format(total))

'Inference complete.'

'Total time elapsed: 0'

In [ ]:
# Using CXR-Large model (224x224 dimensions)

df_out_224 = df_out_lower.copy()

total = 0
ts = int(datetime.datetime.now().timestamp())

weightspath = '/p/project/joaiml/ingolfsson1/COVID_Net/models/COVIDNet-CXR-Large'
metaname = 'model.meta'
ckptname = 'model-8485'
in_tensorname = 'input_1:0'
out_tensorname = 'dense_3/Softmax:0'
input_size = 224

for _, image, cond, source in df_test.values:

    split = 'test'

    if df_out_224[df_out_224['path'] == image]['pred'].iloc[0] == 'unknown':

        complete_path = '/p/project/joaiml/ingolfsson1/COVID_Net/data/{}/{}'.format(split, image)

        t = time.time()
        pred = predict(complete_path, weightspath, metaname, ckptname, in_tensorname, out_tensorname, input_size)
        t = time.time() - t

        total = total + t

        clear_output(wait=True)
        display('Image: {}/{}'.format(split, image))
        display('Expected: {}'.format(cond))
        display('Predicted: {}'.format(pred))
        display('Inference time: {}'.format(t))
        display('Total time elapsed: {}'.format(total))

        df_out_224['pred'] = np.where(df_out_224['path'] == image, pred, df_out_224['pred'])
        df_out_224['time'] = np.where(df_out_224['path'] == image, t, df_out_224['time'])

        df_out_224.to_csv('/p/project/joaiml/ingolfsson1/COVID_Net/results/test_covidx_224_{}.csv'.format(ts), index=False)

clear_output(wait=True)
display('Inference complete.')
display('Total time elapsed: {}'.format(total))

'Image: test/436dce2a-06c3-4281-bb8e-840497a49381.png'

'Expected: normal'

'Predicted: pneumonia'

'Inference time: 7.574856996536255'

'Total time elapsed: 1658.7800579071045'